In [102]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import textwrap
pd.options.display.width = 0
pd.options.display.max_rows = None
pd.options.display.float_format = "{:.2f}".format

'''The task is to check the accounting data with ML'''

'The task is to check the accounting data with ML'

In [103]:
data = pd.read_csv('C:\\Users\\SZ\\Dropbox\\Python Projekt Buchhaltung\\kontobuchungen2018.txt', header=0, sep=';', thousands='.', decimal=',', low_memory=False)

In [104]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362056 entries, 0 to 362055
Data columns (total 39 columns):
Konto                            362056 non-null int64
Belegdatum                       362056 non-null int64
BU-Schlüssel                     52583 non-null float64
Gegenkonto                       362056 non-null int64
Buchungstext                     357709 non-null object
Steuersatz                       362056 non-null float64
Belegfeld 1                      362056 non-null object
Umsatz Soll                      362056 non-null float64
Umsatz Haben                     362056 non-null float64
WKZ eingeg. Umsatz               362056 non-null object
Eingeg. Umsatz                   362056 non-null float64
Kurs                             362056 non-null float64
Stapel-Nr.                       362056 non-null object
BSNr                             362056 non-null int64
Herkunft-Kz                      362056 non-null object
Belegfeld 2                      42 non-null flo

In [105]:
print(data.head())

    Konto  Belegdatum  BU-Schlüssel  Gegenkonto Buchungstext  Steuersatz  \
0   14000     1012018           nan       90080      EB-Wert        0.00   
1   90080     1012018           nan      107000      EB-Wert        0.00   
2  107000     1012018           nan       90080      EB-Wert        0.00   
3   14000     1012018           nan       90080      EB-Wert        0.00   
4   90080     1012018           nan      110000      EB-Wert        0.00   

  Belegfeld 1  Umsatz Soll  Umsatz Haben WKZ eingeg. Umsatz  ...  \
0           0       699.02          0.00                EUR  ...   
1           0         0.00        699.02                EUR  ...   
2           0       699.02          0.00                EUR  ...   
3           0      2250.19          0.00                EUR  ...   
4           0         0.00       2250.19                EUR  ...   

   Gesellschaftername  Beteiligtennummer Identifikationsnummer  \
0                 nan                nan                   nan   
1 

In [106]:
'''Throwing out irrelevant features'''

irrelevant_features = ['WKZ eingeg. Umsatz', 'Stapel-Nr.', 'BSNr', 'Herkunft-Kz', 'Belegfeld 2', 'KOST1 - Kostenstelle', 
                       'KOST2 - Kostenstelle', 'Kost-Menge', 'Stück', 'Gewicht', 'Auftragsnummer', 'EU-Land (Anzahlungen)', 
                       'Sachverhalt L+L (Anzahlungen)', 'Gesellschaftername', 'Beteiligtennummer', 'Identifikationsnummer', 
                       'Zeichnernummer', 'Buchungs GUID', 'Bezeichnung SoBil-Sachverhalt', 'Kennzeichen SoBil-Buchung', 'Leistungsdatum', 'Datum Zuord. Steuerperiode']
data.drop(irrelevant_features, inplace=True, axis=1)

In [107]:
data.head()

,Konto,Belegdatum,BU-Schlüssel,Gegenkonto,Buchungstext,Steuersatz,Belegfeld 1,Umsatz Soll,Umsatz Haben,Eingeg. Umsatz,Kurs,Buchungstyp (Anzahlungen),USt-Schlüssel (Anzahlungen),EU-Steuersatz (Anzahlungen),Erlöskonto (Anzahlungen),Skontotyp,Generalumkehr (GU)
0,14000,1012018,nan,90080,EB-Wert,0.00,0,699.02,0.00,0.00,0.00,nan,0,nan,0,0,0
1,90080,1012018,nan,107000,EB-Wert,0.00,0,0.00,699.02,0.00,0.00,nan,0,nan,0,0,0
2,107000,1012018,nan,90080,EB-Wert,0.00,0,699.02,0.00,0.00,0.00,nan,0,nan,0,0,0
3,14000,1012018,nan,90080,EB-Wert,0.00,0,2250.19,0.00,0.00,0.00,nan,0,nan,0,0,0
4,90080,1012018,nan,110000,EB-Wert,0.00,0,0.00,2250.19,0.00,0.00,nan,0,nan,0,0,0


In [108]:
# data['Konto'].unique()

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362056 entries, 0 to 362055
Data columns (total 17 columns):
Konto                          362056 non-null int64
Belegdatum                     362056 non-null int64
BU-Schlüssel                   52583 non-null float64
Gegenkonto                     362056 non-null int64
Buchungstext                   357709 non-null object
Steuersatz                     362056 non-null float64
Belegfeld 1                    362056 non-null object
Umsatz Soll                    362056 non-null float64
Umsatz Haben                   362056 non-null float64
Eingeg. Umsatz                 362056 non-null float64
Kurs                           362056 non-null float64
Buchungstyp (Anzahlungen)      0 non-null float64
USt-Schlüssel (Anzahlungen)    362056 non-null int64
EU-Steuersatz (Anzahlungen)    0 non-null float64
Erlöskonto (Anzahlungen)       362056 non-null int64
Skontotyp                      362056 non-null int64
Generalumkehr (GU)             362

In [111]:
check_84000_soll = data[(data['Konto'] == 84000) & (data['Umsatz Soll'] > 0)]

In [113]:
check_84000_soll

,Konto,Belegdatum,BU-Schlüssel,Gegenkonto,Buchungstext,Steuersatz,Belegfeld 1,Umsatz Soll,Umsatz Haben,Eingeg. Umsatz,Kurs,Buchungstyp (Anzahlungen),USt-Schlüssel (Anzahlungen),EU-Steuersatz (Anzahlungen),Erlöskonto (Anzahlungen),Skontotyp,Generalumkehr (GU)
1543,84000,10012018,nan,12000,Gschosser Petra,19.00,46614,3.28,0.00,0.00,0.00,nan,0,nan,0,0,0
1546,84000,10012018,nan,12000,Gotzer Fabian,19.00,48325,3.28,0.00,0.00,0.00,nan,0,nan,0,0,0
1549,84000,10012018,nan,12000,Bekci Baris,19.00,36970,6.18,0.00,0.00,0.00,nan,0,nan,0,0,0
1552,84000,10012018,nan,12000,Kathrin Frey,19.00,44132,6.55,0.00,0.00,0.00,nan,0,nan,0,0,0
1555,84000,10012018,nan,12000,Gschosser Petra,19.00,46614,8.32,0.00,0.00,0.00,nan,0,nan,0,0,0
1558,84000,10012018,nan,12000,Marc Lamscheck,19.00,44497,10.92,0.00,0.00,0.00,nan,0,nan,0,0,0
1564,84000,10012018,nan,12000,El Sonbaty Iman Jihan,19.00,21423,11.60,0.00,0.00,0.00,nan,0,nan,0,0,0
1567,84000,10012018,nan,12000,Frau Sandra Csaszar-Bar,19.00,43663,11.76,0.00,0.00,0.00,nan,0,nan,0,0,0
1570,84000,10012018,nan,12000,"Leichtle-Werner, Marion Wer",19.00,39294,16.10,0.00,0.00,0.00,nan,0,nan,0,0,0
1573,84000,10012018,nan,12000,"Cramer, Kay",19.00,39917,17.23,0.00,0.00,0.00,nan,0,nan,0,0,0
